<a href="https://colab.research.google.com/github/23520052/DoAnDuLieuLon/blob/main/2_Baseline_Model_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --- PHẦN 1: CÀI ĐẶT & KẾT NỐI ---
# 1. Cài thư viện cần thiết
!pip install -q transformers torch scikit-learn

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_linear_schedule_with_warmup
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np
import re
import os
import time
from google.colab import drive

# 2. Kiểm tra GPU
if torch.cuda.is_available():
    print(f"✅ ĐÃ BẬT GPU: {torch.cuda.get_device_name(0)}")
    device = torch.device('cuda')
else:
    raise SystemError("❌ LỖI: Chưa bật GPU! Hãy vào Runtime -> Change runtime type -> T4 GPU")

# 3. Kết nối Google Drive
drive.mount('/content/drive')

# --- SỬA ĐƯỜNG DẪN DƯỚI ĐÂY CHO ĐÚNG THƯ MỤC CỦA BẠN ---
folder_path = '/content/drive/My Drive/ABSA_Project'

try:
    os.chdir(folder_path)
    print(f"✅ Đang làm việc tại thư mục: {os.getcwd()}")
except:
    print("⚠️ Không tìm thấy thư mục! Hãy kiểm tra lại biến 'folder_path'.")

# --- PHẦN 2: XỬ LÝ DỮ LIỆU (Parser) ---
def parse_vlsp_data(file_name):
    """Đọc file txt định dạng VLSP 2018"""
    lines = []
    try:
        # Thử đọc utf-16
        with open(file_name, 'r', encoding='utf-16') as f:
            lines = f.readlines()
    except:
        try:
            # Nếu lỗi thì đọc utf-8
            with open(file_name, 'r', encoding='utf-8') as f:
                lines = f.readlines()
        except:
            return pd.DataFrame()

    dataset = []
    review_buffer = []
    # Regex tìm nhãn: {ASPECT, SENTIMENT}
    label_pattern = re.compile(r"\{(.*?),\s*(.*?)\}")

    for line in lines:
        line = line.strip()
        if not line: continue
        if "{" in line and "}" in line and "#" in line:
            current_review = " ".join(review_buffer).strip()
            matches = label_pattern.findall(line)
            for aspect, sentiment in matches:
                if sentiment.lower() != 'none': # Bỏ qua nhãn None
                    dataset.append({
                        'text': current_review,
                        'aspect': aspect.strip(),
                        'label': sentiment.strip().lower()
                    })
            review_buffer = []
        else:
            review_buffer.append(re.sub(r"^[\_\-]\s*", "", line))
    return pd.DataFrame(dataset)

# Đọc dữ liệu
print("⏳ Đang tải dữ liệu...")
df_train = parse_vlsp_data('1-VLSP2018-SA-Restaurant-train.txt')
df_test = parse_vlsp_data('3-VLSP2018-SA-Restaurant-test.txt')
print(f"✅ Train set: {len(df_train)} dòng | Test set: {len(df_test)} dòng")

# --- PHẦN 3: CHUẨN BỊ DATASET CHO PHOBERT ---
print("⏳ Đang tải Tokenizer PhoBERT...")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

class SentimentDataset(Dataset):
    def __init__(self, df, tokenizer, max_len=128):
        self.df = df
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.label_map = {'negative': 0, 'neutral': 1, 'positive': 2}

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        row = self.df.iloc[index]
        # Input format: Aspect [SEP] Review Text
        input_text = f"{row['aspect']} {self.tokenizer.sep_token} {row['text']}"

        encoding = self.tokenizer.encode_plus(
            input_text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(self.label_map[row['label']], dtype=torch.long)
        }

# Tạo DataLoader
BATCH_SIZE = 32 # Tăng lên 32 vì GPU T4 đủ mạnh
train_loader = DataLoader(SentimentDataset(df_train, tokenizer), batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(SentimentDataset(df_test, tokenizer), batch_size=BATCH_SIZE)

# --- PHẦN 4: HUẤN LUYỆN (TRAINING) ---
print("⏳ Đang tải mô hình PhoBERT (Pre-trained)...")
model = AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base", num_labels=3)
model.to(device)

# Cấu hình
EPOCHS = 3
optimizer = AdamW(model.parameters(), lr=2e-5)
total_steps = len(train_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

print(f"\n🚀 BẮT ĐẦU HUẤN LUYỆN TRÊN GPU {torch.cuda.get_device_name(0)}...")
start_time = time.time()

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for step, batch in enumerate(train_loader):
        # Đẩy dữ liệu vào GPU
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # Xử lý
        model.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

        if step % 50 == 0 and step > 0:
            elapsed = time.time() - start_time
            print(f"  Epoch {epoch+1} | Batch {step}/{len(train_loader)} | Loss: {loss.item():.4f} | Time: {elapsed:.0f}s")

# --- PHẦN 5: ĐÁNH GIÁ (EVALUATION) ---
print("\n✅ Đã train xong! Đang chấm điểm trên tập Test...")
model.eval()
predictions, true_labels = [], []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs.logits, dim=1)

        predictions.extend(preds.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# IN BÁO CÁO CUỐI CÙNG
print("\n📊 BẢNG KẾT QUẢ CHI TIẾT (BASELINE - CHƯA CÂN BẰNG):")
target_names = ['Negative', 'Neutral', 'Positive']
print(classification_report(true_labels, predictions, target_names=target_names, digits=4, zero_division=0))